Torch version check
--

In [1]:
import torch
print(torch.__version__)

2.2.2


In [30]:
!df -h

The history saving thread hit an unexpected error (OperationalError('unable to open database file')).History will not be written to the database.
Filesystem      Size  Used Avail Use% Mounted on
devtmpfs         32G     0   32G   0% /dev
tmpfs            32G   96K   32G   1% /dev/shm
tmpfs            32G  840K   32G   1% /run
tmpfs            32G     0   32G   0% /sys/fs/cgroup
/dev/nvme0n1p1  135G  135G   16K 100% /
tmpfs           6.3G     0  6.3G   0% /run/user/0
/dev/nvme2n1    985G  1.6G  933G   1% /home/ec2-user/SageMaker
tmpfs           6.3G     0  6.3G   0% /run/user/1001
tmpfs           6.3G     0  6.3G   0% /run/user/1002
tmpfs           6.3G     0  6.3G   0% /run/user/1000


Installing package datasets
--

In [2]:
!pip install datasets

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.6/1.6 MB 76.0 MB/s eta 0:00:00
  Attempting uninstall: fsspec
    Found existing installation: fsspec 2024.10.0
    Uninstalling fsspec-2024.10.0:
      Successfully uninstalled fsspec-2024.10.0
  Attempting uninstall: dill
    Found existing installation: dill 0.3.9
    Uninstalling dill-0.3.9:
      Successfully uninstalled dill-0.3.9
  Attempting uninstall: multiprocess
    Found existing installation: multiprocess 0.70.17
    Uninstalling multiprocess-0.70.17:
      Successfully uninstalled multiprocess-0.70.17
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
pathos 0.3.3 requires dill>=0.3.9, but you have dill 0.3.8 which is incompatible.
pathos 0.3.3 requires multiprocess>=0.70.17, but you have multiprocess 0.70.16 which is incompatible.


Installing package evaluate
--

In [3]:
!pip install evaluate

Installing package accelerate
--

In [4]:
!pip install accelerate

Installing package peft
--

In [5]:
!pip install peft

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 10.1/10.1 MB 104.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 781.7/781.7 kB 44.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.0/3.0 MB 122.7 MB/s eta 0:00:00


In [6]:
!pip install bitsandbytes \ trl \

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 69.1/69.1 MB 132.0 MB/s eta 0:00:0000:0100:01


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 10.0/10.0 MB 128.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.0/3.0 MB 101.3 MB/s eta 0:00:00
  Attempting uninstall: tokenizers
    Found existing installation: tokenizers 0.21.0
    Uninstalling tokenizers-0.21.0:
      Successfully uninstalled tokenizers-0.21.0
  Attempting uninstall: transformers
    Found existing installation: transformers 4.47.0
    Uninstalling transformers-4.47.0:
      Successfully uninstalled transformers-4.47.0


In [7]:
!pip install datasets \
                evaluate \
                loralib \
                accelerate \
                bitsandbytes \
                trl \
                peft

In [8]:
import peft
print(peft.__version__)

0.14.0


In [9]:
# import peft
# dir(peft)


In [10]:
!pip install soundfile

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 50.1 MB/s eta 0:00:00


In [11]:
!pip install librosa soundfile

In [12]:
!pip install jiwer

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.1/3.1 MB 75.4 MB/s eta 0:00:00


Importing packages
--


In [13]:
from datasets import load_dataset, DatasetDict
from transformers import (
    WhisperTokenizer,
    WhisperProcessor,
    WhisperFeatureExtractor,
    WhisperForConditionalGeneration,
    Seq2SeqTrainingArguments,
    Seq2SeqTrainer,
    TrainerCallback,
    logging
)
from datasets import Audio
from torch.utils.data import DataLoader
import torch
import evaluate
from accelerate import Accelerator
from typing import Any, Dict, List, Union
# from peft import prepare_model_for_int8_training  # If it's in the main module


In [14]:
import os
os.listdir('whisper_output_dir_10/checkpoint-520')

['optimizer.pt',
 'preprocessor_config.json',
 'adapter_model',
 'scheduler.pt',
 'adapter_config.json',
 'README.md',
 'rng_state.pth',
 'trainer_state.json',
 'adapter_model.safetensors',
 'training_args.bin']

Importing model
--

In [15]:
# model_id='openai/whisper-large-v3-turbo'
model_id = "/home/ec2-user/SageMaker/whisper_output_dir_10/checkpoint-1000/"
os.listdir(model_id)

['optimizer.pt',
 'preprocessor_config.json',
 'adapter_model',
 'scheduler.pt',
 'adapter_config.json',
 'README.md',
 'rng_state.pth',
 'trainer_state.json',
 'adapter_model.safetensors',
 'training_args.bin']

In [16]:
os.getcwd()

'/home/ec2-user/SageMaker'

Loading dataset
--

In [17]:
from datasets import load_from_disk, concatenate_datasets

# dataset = load_from_disk("s3://stt-jordan-dataset/jordan_dataset_v1/")
# atc_dataset_train = dataset['train']
# atc_dataset_valid = dataset['validation']
# atc_dataset_test = dataset['test']

# atc_dataset_train = load_dataset('crtvai/jordan-dataset-v3', split='train')
# atc_dataset_valid = load_dataset('crtvai/jordan-dataset-v3', split='validation')
# atc_dataset_test = load_dataset('crtvai/jordan-dataset-v3', split='test')

In [18]:
stt_data_11 = load_dataset('crtvai/jordan-dataset-v11')
stt_data_12 = load_dataset('crtvai/jordan-dataset-v12')
stt_data_13 = load_dataset('crtvai/jordan-dataset-v13')
stt_data_14 = load_dataset('crtvai/jordan-dataset-v14')
stt_data_15 = load_dataset('crtvai/jordan-dataset-v15')
stt_data_16 = load_dataset('crtvai/jordan-dataset-v16')
stt_data_17 = load_dataset('crtvai/jordan-dataset-v17')
stt_data_18 = load_dataset('crtvai/jordan-dataset-v18')
stt_data_19 = load_dataset('crtvai/jordan-dataset-v19')
stt_data_20 = load_dataset('crtvai/jordan-dataset-v20')
# stt_data_21 = load_dataset('crtvai/jordan-dataset-v21')
# stt_data_22 = load_dataset('crtvai/jordan-dataset-v22')
# stt_data_23 = load_dataset('crtvai/jordan-dataset-v23')
# stt_data_24 = load_dataset('crtvai/jordan-dataset-v24')
# stt_data_25 = load_dataset('crtvai/jordan-dataset-v25')
# stt_data_26 = load_dataset('crtvai/jordan-dataset-v26')
# stt_data_27 = load_dataset('crtvai/jordan-dataset-v27')
# stt_data_28 = load_dataset('crtvai/jordan-dataset-v28')
# stt_data_29 = load_dataset('crtvai/jordan-dataset-v29')
# stt_data_30 = load_dataset('crtvai/jordan-dataset-v30')



atc_dataset_train = concatenate_datasets([stt_data_11['train'], stt_data_12['train'],stt_data_13['train'],
                                          stt_data_14['train'],stt_data_15['train'],stt_data_16['train'],
                                          stt_data_17['train'],stt_data_18['train'],stt_data_19['train'],
                                          stt_data_20['train']])

atc_dataset_valid = concatenate_datasets([stt_data_11['validation'], stt_data_12['validation'],stt_data_13['validation'],
                                          stt_data_14['validation'],stt_data_15['validation'],stt_data_16['validation'],
                                          stt_data_17['validation'],stt_data_18['validation'],stt_data_19['validation'],
                                          stt_data_20['validation']])

atc_dataset_test = concatenate_datasets([stt_data_11['test'], stt_data_12['test'],stt_data_13['test'],stt_data_14['test'],
                                         stt_data_15['test'],stt_data_16['test'],stt_data_17['test'],stt_data_18['test'],
                                         stt_data_19['test'],stt_data_20['test']])

print(atc_dataset_train, atc_dataset_valid, atc_dataset_test)

README.md:   0%|          | 0.00/591 [00:00<?, ?B/s]

train-00000-of-00001.parquet:   0%|          | 0.00/134M [00:00<?, ?B/s]

test-00000-of-00001.parquet:   0%|          | 0.00/18.4M [00:00<?, ?B/s]

validation-00000-of-00001.parquet:   0%|          | 0.00/16.7M [00:00<?, ?B/s]

Generating train split:   0%|          | 0/122 [00:00<?, ? examples/s]

...

validation-00000-of-00001.parquet:   0%|          | 0.00/212M [00:00<?, ?B/s]

Generating train split:   0%|          | 0/1594 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/200 [00:00<?, ? examples/s]

Generating validation split:   0%|          | 0/199 [00:00<?, ? examples/s]

Dataset({
    features: ['audio', 'text'],
    num_rows: 12614
}) Dataset({
    features: ['audio', 'text'],
    num_rows: 1575
}) Dataset({
    features: ['audio', 'text'],
    num_rows: 1582
})


In [19]:
atc_dataset_train

Dataset({
    features: ['audio', 'text'],
    num_rows: 12614
})

### Get Sampling rate of Audio

In [20]:
atc_dataset_train[0]

{'audio': {'path': 'stt-dataset-v11_161400_164560.wav',
  'array': array([-0.00251552, -0.00150318,  0.00389309, ..., -0.02810975,
         -0.02786456, -0.03060853]),
  'sampling_rate': 16000},
 'text': 'بالبهارات وايضا بقدر اربط لك طريقه غش في'}

In [21]:
# model_id = "mohammed/whisper-large-arabic-cv-11"
# dataset_cc = concatenate_datasets([dataset_ar['train'], dataset_bn['train']])
model_id = 'whisper_output_dir_10/checkpoint-1000'
from transformers import WhisperFeatureExtractor

feature_extractor = WhisperFeatureExtractor.from_pretrained(model_id)

In [27]:
!df -h

Filesystem      Size  Used Avail Use% Mounted on
devtmpfs         32G     0   32G   0% /dev
tmpfs            32G   24K   32G   1% /dev/shm
tmpfs            32G  712K   32G   1% /run
tmpfs            32G     0   32G   0% /sys/fs/cgroup
/dev/nvme0n1p1  135G  116G   20G  86% /
tmpfs           6.3G     0  6.3G   0% /run/user/0
/dev/nvme2n1    985G  1.6G  933G   1% /home/ec2-user/SageMaker
tmpfs           6.3G     0  6.3G   0% /run/user/1001
tmpfs           6.3G     0  6.3G   0% /run/user/1002
tmpfs           6.3G     0  6.3G   0% /run/user/1000


In [22]:
openai_turbo = "openai/whisper-large-v3-turbo"
tokenizer = WhisperTokenizer.from_pretrained(openai_turbo, language='Arabic', task='transcribe')
processor = WhisperProcessor.from_pretrained(openai_turbo, language='Arabic', task='transcribe')

tokenizer_config.json:   0%|          | 0.00/283k [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/1.04M [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/2.71M [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/494k [00:00<?, ?B/s]

normalizer.json:   0%|          | 0.00/52.7k [00:00<?, ?B/s]

added_tokens.json:   0%|          | 0.00/34.6k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/2.19k [00:00<?, ?B/s]

preprocessor_config.json:   0%|          | 0.00/340 [00:00<?, ?B/s]

In [23]:
atc_dataset_train = atc_dataset_train.cast_column('audio', Audio(sampling_rate=16000))
atc_dataset_valid = atc_dataset_valid.cast_column('audio', Audio(sampling_rate=16000))
atc_dataset_test = atc_dataset_test.cast_column('audio', Audio(sampling_rate=16000))

https://huggingface.co/mohammed/whisper-large-arabic-cv-11

In [24]:
# get the processor and model from mohammed/whisper-small-arabic-cv-11
# processor = WhisperProcessor.from_pretrained("mohammed/whisper-large-arabic-cv-11")
# model = WhisperForConditionalGeneration.from_pretrained("mohammed/whisper-large-arabic-cv-11")
# model.config.forced_decoder_ids = None
# # 
# resample the audio files to 16000
# test_dataset = test_dataset.cast_column("audio", Audio(sampling_rate=16000))


In [25]:
def prepare_dataset(batch):
    audio = batch['audio']
    batch['input_features'] = feature_extractor(audio['array'], sampling_rate=audio['sampling_rate']).input_features[0]
    batch['labels'] = tokenizer(batch['text']).input_ids
    return batch

In [28]:
!df -h



Filesystem      Size  Used Avail Use% Mounted on
devtmpfs         32G     0   32G   0% /dev
tmpfs            32G   28K   32G   1% /dev/shm
tmpfs            32G  712K   32G   1% /run
tmpfs            32G     0   32G   0% /sys/fs/cgroup
/dev/nvme0n1p1  135G  116G   20G  86% /
tmpfs           6.3G     0  6.3G   0% /run/user/0
/dev/nvme2n1    985G  1.6G  933G   1% /home/ec2-user/SageMaker
tmpfs           6.3G     0  6.3G   0% /run/user/1001
tmpfs           6.3G     0  6.3G   0% /run/user/1002
tmpfs           6.3G     0  6.3G   0% /run/user/1000


In [29]:
atc_dataset_train = atc_dataset_train.map(prepare_dataset, num_proc=4)
atc_dataset_valid = atc_dataset_valid.map(prepare_dataset, num_proc=4)
atc_dataset_test = atc_dataset_test.map(prepare_dataset, num_proc=4)

Map (num_proc=4):   0%|          | 0/12614 [00:00<?, ? examples/s]

Map (num_proc=4):   0%|          | 0/1575 [00:00<?, ? examples/s]

OSError: [Errno 28] No space left on device

In [ ]:
metric = evaluate.load('wer')
def compute_metrics(pred):
    pred_ids = pred.predictions
    label_ids = pred.label_ids
    label_ids[label_ids == -100] = tokenizer.pad_token_id
    pred_str = tokenizer.batch_decode(pred_ids, skip_special_tokens=True)
    label_str = tokenizer.batch_decode(label_ids, skip_special_tokens=True)
    wer = 100 * metric.compute(predictions=pred_str, references=label_str)
    return {'wer': wer}


In [ ]:
class DataCollatorSpeechSeq2SeqWithPadding:
    def __init__(self, processor: Any, decoder_start_token_id: int):
        self.processor = processor
        self.decoder_start_token_id = decoder_start_token_id

    def __call__(self, features: List[Dict[str, Union[List[int], torch.Tensor]]]) -> Dict[str, torch.Tensor]:
        input_features = [{'input_features': feature['input_features']} for feature in features]
        batch = self.processor.feature_extractor.pad(input_features, return_tensors='pt')
        label_features = [{'input_ids': feature['labels']} for feature in features]
        labels_batch = self.processor.tokenizer.pad(label_features, return_tensors='pt')
        labels = labels_batch['input_ids'].masked_fill(labels_batch.attention_mask.ne(1), -100)
        if (labels[:, 0] == self.decoder_start_token_id).all().cpu().item():
            labels = labels[:, 1:]
        batch['labels'] = labels
        return batch

In [ ]:
model_id = "whisper_output_dir_10/checkpoint-1000"
model = WhisperForConditionalGeneration.from_pretrained(model_id, load_in_8bit=True, device_map="auto")
model.generation_config.task = 'transcribe'
model.generation_config.forced_decoder_ids = None

In [ ]:
data_collator = DataCollatorSpeechSeq2SeqWithPadding(
        processor=processor,
        decoder_start_token_id=model.config.decoder_start_token_id,
    )

In [47]:
# from peft import prepare_model_for_int8_training
# model = prepare_model_for_int8_training(model, output_embedding_layer_name="proj_out")

In [ ]:
def make_inputs_require_grad(module, input, output):
    output.requires_grad_(True)

model.model.encoder.conv1.register_forward_hook(make_inputs_require_grad)

In [ ]:
from peft import LoraConfig, PeftModel, LoraModel, LoraConfig, get_peft_model

config = LoraConfig(r=32, lora_alpha=64, target_modules=["q_proj", "v_proj"], lora_dropout=0.05, bias="none")

model = get_peft_model(model, config)
model.print_trainable_parameters()

In [ ]:
from transformers import Seq2SeqTrainingArguments


# output_dir = "/home/ec2-user/SageMaker/whisper_output_dir"
training_args = Seq2SeqTrainingArguments(
    output_dir="whisper_output_dir_11",  # change to a repo name of your choice
    per_device_train_batch_size=1,
    per_device_eval_batch_size=1,
#     evaluation_batch_size = 1,
    gradient_accumulation_steps=32,  # increase by 2x for every 2x decrease in batch size
    learning_rate=0.00001,
    warmup_steps=1000,
    num_train_epochs=10,
    evaluation_strategy="steps",
    fp16=True,
    generation_max_length=128,
    logging_steps=100,
    dataloader_num_workers=2,
    lr_scheduler_type='constant',
    seed=42,
    data_seed=42,
#    max_steps=100, # only for testing purposes, remove this from your final run :)
    remove_unused_columns=False,  # required as the PeftModel forward doesn't have the signature of the wrapped model's forward
    label_names=["labels"],  # same reason as above
)

In [ ]:
from transformers import Seq2SeqTrainer, TrainerCallback, TrainingArguments, TrainerState, TrainerControl
from transformers.trainer_utils import PREFIX_CHECKPOINT_DIR
import os
# This callback helps to save only the adapter weights and remove the base model weights.
class SavePeftModelCallback(TrainerCallback):
    def on_save(
        self,
        args: TrainingArguments,
        state: TrainerState,
        control: TrainerControl,
        **kwargs,
    ):
        checkpoint_folder = os.path.join(args.output_dir, f"{PREFIX_CHECKPOINT_DIR}-{state.global_step}")

        peft_model_path = os.path.join(checkpoint_folder, "adapter_model")
        kwargs["model"].save_pretrained(peft_model_path)

        pytorch_model_path = os.path.join(checkpoint_folder, "pytorch_model.bin")
        if os.path.exists(pytorch_model_path):
            os.remove(pytorch_model_path)
        return control


trainer = Seq2SeqTrainer(
    args=training_args,
    model=model,
    train_dataset=atc_dataset_train,
    eval_dataset=atc_dataset_valid,
    data_collator=data_collator,
    tokenizer=processor.feature_extractor,
    callbacks=[SavePeftModelCallback],
)
model.config.use_cache = False 


### Training arguments for turbo

In [83]:
training_args = Seq2SeqTrainingArguments(
    output_dir='whisper_output_dir_2',
    per_device_train_batch_size=1,
    per_device_eval_batch_size=1,
    gradient_accumulation_steps=32,
    learning_rate=0.00001,
    warmup_steps=1000,
    fp16=True,
    num_train_epochs=3,
    evaluation_strategy='epoch',
    logging_strategy='epoch',
    save_strategy='epoch',
    predict_with_generate=True,
    generation_max_length=225,
    load_best_model_at_end=True,
    metric_for_best_model='wer',
    greater_is_better=False,
    dataloader_num_workers=2,
    save_total_limit=2,
    lr_scheduler_type='constant',
    seed=42,
    data_seed=42,
  )

/home/ec2-user/anaconda3/envs/pytorch_p310/lib/python3.10/site-packages/transformers/training_args.py:1568: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(


In [51]:
# Metric for WER
metric = evaluate.load('wer')
def compute_metrics(pred):
  pred_ids = pred.predictions
  label_ids = pred.label_ids
  label_ids[label_ids == -100] = tokenizer.pad_token_id
  pred_str = tokenizer.batch_decode(pred_ids, skip_special_tokens=True)
  label_str = tokenizer.batch_decode(label_ids, skip_special_tokens=True)
  wer = 100 * metric.compute(predictions=pred_str, references=label_str)
  return {'wer': wer}

# Callback to clear cache during training
class ClearCacheCallback(TrainerCallback):
  def on_step_end(self, args, state, control, model=None, tokenizer=None, **kwargs):
    torch.cuda.empty_cache()
    gc.collect()

trainer = Seq2SeqTrainer(
    args=training_args,
    model=model,
    train_dataset=atc_dataset_train,
    eval_dataset=atc_dataset_valid,
    data_collator=data_collator,
    compute_metrics=compute_metrics,
    callbacks=[ClearCacheCallback()],
  )

In [ ]:
trainer.train()

/home/ec2-user/anaconda3/envs/pytorch_p310/lib/python3.10/site-packages/bitsandbytes/autograd/_functions.py:316: UserWarning: MatMul8bitLt: inputs will be cast from torch.float32 to float16 during quantization
  warnings.warn(f"MatMul8bitLt: inputs will be cast from {A.dtype} to float16 during quantization")


Step,Training Loss,Validation Loss


TrainOutput(global_step=90, training_loss=0.9477957831488715, metrics={'train_runtime': 1490.9822, 'train_samples_per_second': 1.98, 'train_steps_per_second': 0.06, 'total_flos': 4.9537458634752e+18, 'train_loss': 0.9477957831488715, 'epoch': 2.926829268292683})

In [18]:
trainer.train()

/home/ec2-user/anaconda3/envs/pytorch_p310/lib/python3.10/site-packages/bitsandbytes/autograd/_functions.py:316: UserWarning: MatMul8bitLt: inputs will be cast from torch.float32 to float16 during quantization
  warnings.warn(f"MatMul8bitLt: inputs will be cast from {A.dtype} to float16 during quantization")


Step,Training Loss,Validation Loss
100,1.033100,0.741907
200,0.587100,0.513101
300,0.447800,0.439277


TrainOutput(global_step=300, training_loss=0.6893380482991537, metrics={'train_runtime': 5027.7968, 'train_samples_per_second': 1.957, 'train_steps_per_second': 0.06, 'total_flos': 1.6512486211584e+19, 'train_loss': 0.6893380482991537, 'epoch': 9.75609756097561})

In [ ]:
trainer.train()

/home/ec2-user/anaconda3/envs/pytorch_p310/lib/python3.10/site-packages/bitsandbytes/autograd/_functions.py:316: UserWarning: MatMul8bitLt: inputs will be cast from torch.float32 to float16 during quantization
  warnings.warn(f"MatMul8bitLt: inputs will be cast from {A.dtype} to float16 during quantization")


Step,Training Loss,Validation Loss
100,0.967200,0.703965
200,0.521600,0.504370
300,0.406300,0.442373


TrainOutput(global_step=370, training_loss=0.5791569374703073, metrics={'train_runtime': 6177.0971, 'train_samples_per_second': 1.93, 'train_steps_per_second': 0.06, 'total_flos': 2.03653996609536e+19, 'train_loss': 0.5791569374703073, 'epoch': 9.93288590604027})

In [100]:
trainer.train()

/home/ec2-user/anaconda3/envs/pytorch_p310/lib/python3.10/site-packages/bitsandbytes/autograd/_functions.py:316: UserWarning: MatMul8bitLt: inputs will be cast from torch.float32 to float16 during quantization
  warnings.warn(f"MatMul8bitLt: inputs will be cast from {A.dtype} to float16 during quantization")


Step,Training Loss,Validation Loss
100,1.167400,0.897582
200,0.702100,0.660559
300,0.576900,0.591013
400,0.513900,0.553206
500,0.460900,0.533319


/home/ec2-user/anaconda3/envs/pytorch_p310/lib/python3.10/site-packages/bitsandbytes/autograd/_functions.py:316: UserWarning: MatMul8bitLt: inputs will be cast from torch.float32 to float16 during quantization
  warnings.warn(f"MatMul8bitLt: inputs will be cast from {A.dtype} to float16 during quantization")


TrainOutput(global_step=520, training_loss=0.6755305895438561, metrics={'train_runtime': 8805.9516, 'train_samples_per_second': 1.915, 'train_steps_per_second': 0.059, 'total_flos': 2.86216427667456e+19, 'train_loss': 0.6755305895438561, 'epoch': 9.869513641755635})

In [ ]:
trainer.train()

/home/ec2-user/anaconda3/envs/pytorch_p310/lib/python3.10/site-packages/bitsandbytes/autograd/_functions.py:315: UserWarning: MatMul8bitLt: inputs will be cast from torch.float32 to float16 during quantization
  warnings.warn(f"MatMul8bitLt: inputs will be cast from {A.dtype} to float16 during quantization")


Step,Training Loss,Validation Loss
100,1.095200,0.864919
200,0.680900,0.624944


In [ ]:
trainer.train()

Calculate the word error rate first (Wer)
--

In [ ]:
# from peft import PeftModel, PeftConfig
# from transformers import WhisperForConditionalGeneration, Seq2SeqTrainer

# peft_model_id = "reach-vb/whisper-large-v2-hindi-100steps" # Use the same model ID as before.
# peft_config = PeftConfig.from_pretrained(peft_model_id)
# model = WhisperForConditionalGeneration.from_pretrained(
#     peft_config.base_model_name_or_path, load_in_8bit=True, device_map="auto"
# )
# model = PeftModel.from_pretrained(model, peft_model_id)
# model.config.use_cache = True

### WER for genuine OPENAI Turbo

In [52]:


import gc
import numpy as np
from tqdm import tqdm
from torch.utils.data import DataLoader
from transformers.models.whisper.english_normalizer import BasicTextNormalizer


language = 'Arabic'
task = 'transcribe'
eval_dataloader = DataLoader(atc_dataset_test, batch_size=1, collate_fn=data_collator)
forced_decoder_ids = processor.get_decoder_prompt_ids(language=language, task=task)
normalizer = BasicTextNormalizer()

predictions = []
references = []
normalized_predictions = []
normalized_references = []

model.eval()
for step, batch in enumerate(tqdm(eval_dataloader)):
    with torch.cuda.amp.autocast():
        with torch.no_grad():
            generated_tokens = (
                model.generate(
                    input_features=batch["input_features"].to("cuda"),
                    forced_decoder_ids=forced_decoder_ids,
                    max_new_tokens=255,
                )
                .cpu()
                .numpy()
            )
            labels = batch["labels"].cpu().numpy()
            labels = np.where(labels != -100, labels, processor.tokenizer.pad_token_id)
            decoded_preds = processor.tokenizer.batch_decode(generated_tokens, skip_special_tokens=True)
            decoded_labels = processor.tokenizer.batch_decode(labels, skip_special_tokens=True)
            predictions.extend(decoded_preds)
            references.extend(decoded_labels)
            normalized_predictions.extend([normalizer(pred).strip() for pred in decoded_preds])
            normalized_references.extend([normalizer(label).strip() for label in decoded_labels])
        del generated_tokens, labels, batch
    gc.collect()
wer = 100 * metric.compute(predictions=predictions, references=references)
normalized_wer = 100 * metric.compute(predictions=normalized_predictions, references=normalized_references)
eval_metrics = {"eval/wer": wer, "eval/normalized_wer": normalized_wer}

print(f"{wer=} and {normalized_wer=}")
print(eval_metrics)

  0%|          | 0/123 [00:00<?, ?it/s]/home/ec2-user/anaconda3/envs/pytorch_p310/lib/python3.10/site-packages/bitsandbytes/autograd/_functions.py:315: UserWarning: MatMul8bitLt: inputs will be cast from torch.float32 to float16 during quantization
  warnings.warn(f"MatMul8bitLt: inputs will be cast from {A.dtype} to float16 during quantization")
100%|██████████| 123/123 [01:53<00:00,  1.08it/s]

wer=91.66666666666666 and normalized_wer=89.55399061032864
{'eval/wer': 91.66666666666666, 'eval/normalized_wer': 89.55399061032864}


#### Whisper Genuine turbo v3
wer=91.66666666666666 and normalized_wer=89.55399061032864

{'eval/wer': 91.66666666666666, 'eval/normalized_wer': 89.55399061032864}
#### fine tuned turbo v3
wer=26.985559566787003 and normalized_wer=26.624548736462096

{'eval/wer': 26.985559566787003, 'eval/normalized_wer': 26.624548736462096}


In [ ]:
import gc
import numpy as np
from tqdm import tqdm
from torch.utils.data import DataLoader
from transformers.models.whisper.english_normalizer import BasicTextNormalizer

language = 'Arabic'
task = 'transcribe'
eval_dataloader = DataLoader(atc_dataset_test, batch_size=1, collate_fn=data_collator)
forced_decoder_ids = processor.get_decoder_prompt_ids(language=language, task=task)
normalizer = BasicTextNormalizer()

predictions = []
references = []
normalized_predictions = []
normalized_references = []

model.eval()
for step, batch in enumerate(tqdm(eval_dataloader)):
    with torch.cuda.amp.autocast():
        with torch.no_grad():
            generated_tokens = (
                model.generate(
                    input_features=batch["input_features"].to("cuda"),
                    forced_decoder_ids=forced_decoder_ids,
                    max_new_tokens=255,
                )
                .cpu()
                .numpy()
            )
            labels = batch["labels"].cpu().numpy()
            labels = np.where(labels != -100, labels, processor.tokenizer.pad_token_id)
            decoded_preds = processor.tokenizer.batch_decode(generated_tokens, skip_special_tokens=True)
            decoded_labels = processor.tokenizer.batch_decode(labels, skip_special_tokens=True)
            predictions.extend(decoded_preds)
            references.extend(decoded_labels)
            normalized_predictions.extend([normalizer(pred).strip() for pred in decoded_preds])
            normalized_references.extend([normalizer(label).strip() for label in decoded_labels])
        del generated_tokens, labels, batch
    gc.collect()
wer = 100 * metric.compute(predictions=predictions, references=references)
normalized_wer = 100 * metric.compute(predictions=normalized_predictions, references=normalized_references)
eval_metrics = {"eval/wer": wer, "eval/normalized_wer": normalized_wer}

print(f"{wer=} and {normalized_wer=}")
print(eval_metrics)

In [ ]:
# wer=26.985559566787003 and normalized_wer=26.624548736462096
# {'eval/wer': 26.985559566787003, 'eval/normalized_wer': 26.624548736462096}

# wer=35.56910569105691 and normalized_wer=35.230352303523034
# {'eval/wer': 35.56910569105691, 'eval/normalized_wer': 35.230352303523034}

# wer=62.558685446009385 and normalized_wer=61.737089201877936
# {'eval/wer': 62.558685446009385, 'eval/normalized_wer': 61.737089201877936}

In [72]:
atc_dataset_test

Dataset({
    features: ['audio', 'text', 'input_features', 'labels'],
    num_rows: 150
})

In [76]:
atc_dataset_test = load_dataset('crtvai/jordan-dataset-v2', split='test')

path = atc_dataset_test[0]['audio']['path']
original_text = atc_dataset_test[0]['text']
print(atc_dataset_test[0], path, original_text)

{'audio': {'path': 'stt-dataset-v2_1185799_1189919.wav', 'array': array([ 0.00905005,  0.00897712, -0.0005324 , ..., -0.20471674,
       -0.17537695, -0.1864945 ]), 'sampling_rate': 16000}, 'text': 'واعط وحطيت رقم التلفون رقم التلفون لما'} stt-dataset-v2_1185799_1189919.wav واعط وحطيت رقم التلفون رقم التلفون لما


In [79]:
from datasets import load_dataset, DatasetDict



# Save the dataset to a local directory
atc_dataset_test.save_to_disk("test_data")

# Load the dataset back from the local directory
# loaded_dataset = DatasetDict.load_from_disk("./local_hf_data/ag_news_train")

# Verify loaded dataset
# print("Loaded dataset:", loaded_dataset)


Saving the dataset (0/1 shards):   0%|          | 0/150 [00:00<?, ? examples/s]

In [80]:
result = "واعط وحطيت رقم التلفون رقم التلفون لما"

In [21]:
atc_dataset_train[1]

{'audio': {'path': 'stt-dataset-v1_3394799_3399359.wav',
  'array': array([-0.01602129, -0.00094746, -0.03461099, ..., -0.01498899,
         -0.01418713, -0.01618887]),
  'sampling_rate': 16000},
 'text': 'الشقه عشان اجرها مجدي ولا لا هذا الموضوع'}

### Inference

In [ ]:
import torch
from transformers import AutoModelForSpeechSeq2Seq, AutoProcessor, pipeline
from datasets import load_dataset


device = "cuda:0" if torch.cuda.is_available() else "cpu"
torch_dtype = torch.float16 if torch.cuda.is_available() else torch.float32

model_id = "openai/whisper-large-v3-turbo"

model = AutoModelForSpeechSeq2Seq.from_pretrained(
    model_id, torch_dtype=torch_dtype, low_cpu_mem_usage=True, use_safetensors=True
)
model.to(device)

processor = AutoProcessor.from_pretrained(model_id, language='Arabic',task='transcribe')

pipe = pipeline(
    "automatic-speech-recognition",
    model=model,
    tokenizer=processor.tokenizer,
    feature_extractor=processor.feature_extractor,
    torch_dtype=torch_dtype,
    device=device,
)


def official_transcribe(audio):
    result = pipe(audio)
    return result["text"]


In [58]:
transcribe(atc_dataset_train[1]['audio'])

Due to a bug fix in https://github.com/huggingface/transformers/pull/28687 transcription using a multilingual Whisper will default to language detection followed by transcription instead of translation to English.This might be a breaking change for your use case. If you want to instead always translate your audio to English, make sure to pass `language='en'`.
Passing a tuple of `past_key_values` is deprecated and will be removed in Transformers v4.43.0. You should pass an instance of `EncoderDecoderCache` instead, e.g. `past_key_values=EncoderDecoderCache.from_legacy_cache(past_key_values)`.


' هذا الموضوع'

In [60]:
for i in range(60,80):
    data = atc_dataset_test[i]
    print(f"Real Output     : {data['text']}")
    print(f"Predicted Output: {transcribe(data['audio'])}")
    print('')

Real Output     : عمان شرق ولا غرب ولا شمال ولا جن كل
Predicted Output:  لا يوجد عمان شرق ولا غرب ولا شمال ولا جنة

Real Output     : تروح تشوف العقار الصبح وبالليل وتروح
Predicted Output:  تروح تشوف العقار الصبح وبالليلة وتروح

Real Output     : خمسة و عشرون لانه لما تقسمها على سنتين بتكون
Predicted Output:  25% لأنه ده ما تقسمها على سنتين بتكون

Real Output     : القطاع ونشر محتوى صحي وحقيقي عن السوق
Predicted Output:  ونشر محتوى صحي وحقيقي عن السلطة

Real Output     : الاراضي داخل التنظيم اقل بكثير من
Predicted Output:  داخل التنظيم أقل بكثير من

Real Output     : زي اي شقه لكن موجوده تحت مستوى منسوب
Predicted Output:  لكن موجودة تحت مستوى منصوب الشعب

Real Output     : الدخل فاكيد بتوجهه لحلول التمويل عننا
Predicted Output:  فأكيد بتوجهه لحلول التمويل عندما

Real Output     : الخدمات المرتبطه بالعقار قانونيه لي
Predicted Output:  الخدمات المرتبطة بالعقار قانونية لإلي

Real Output     : هيك النسبه يعني هذا الارقام عاليه جدا
Predicted Output:  مش هيكي نسبة أرقام عالية جداً

Real Outp

In [ ]:
from transformers import (
    AutomaticSpeechRecognitionPipeline,
    WhisperForConditionalGeneration,
    WhisperTokenizer,
    WhisperProcessor,
)
from peft import PeftModel, PeftConfig

peft_model_id = "/home/ec2-user/SageMaker/whisper_output_dir_10/checkpoint-520/"
# peft_model_id = "openai/whisper-large-v3-turbo"
# peft_model_id = "reach-vb/whisper-large-v2-hindi-100steps" # Use the same model ID as before.
language = 'Arabic'
task = 'transcribe'
peft_config = PeftConfig.from_pretrained(peft_model_id)
model = WhisperForConditionalGeneration.from_pretrained(
    peft_config.base_model_name_or_path, load_in_8bit=True, device_map="auto"
)

model = PeftModel.from_pretrained(model, peft_model_id)
tokenizer = WhisperTokenizer.from_pretrained(peft_config.base_model_name_or_path, language=language, task=task)
processor = WhisperProcessor.from_pretrained(peft_config.base_model_name_or_path, language=language, task=task)
feature_extractor = processor.feature_extractor
forced_decoder_ids = processor.get_decoder_prompt_ids(language=language, task=task)
pipe = AutomaticSpeechRecognitionPipeline(model=model, tokenizer=tokenizer, feature_extractor=feature_extractor)


def transcribe(audio):
    with torch.cuda.amp.autocast():
        text = pipe(audio, generate_kwargs={"forced_decoder_ids": forced_decoder_ids}, max_new_tokens=255)["text"]
    return text

transcribe(atc_dataset_train[1]['audio'])

In [62]:
for i in range(60,80):
    data = atc_dataset_test[i]
    print(f"Real Output     : {data['text']}")
    print(f"Predicted Output: {transcribe(data['audio'])}")
    print('')

Real Output     : عمان شرق ولا غرب ولا شمال ولا جن كل
Predicted Output: عمان شرق ولا غرب ولا شمال ولا جنه كل

Real Output     : تروح تشوف العقار الصبح وبالليل وتروح
Predicted Output: تروح تشوف العقار الصبح وبالليله وتروح

Real Output     : خمسة و عشرون لانه لما تقسمها على سنتين بتكون
Predicted Output: خمسه وعشرون بالميه لانه دم تقسمها على سنتين بتكون

Real Output     : القطاع ونشر محتوى صحي وحقيقي عن السوق
Predicted Output: القطاع ونشر محتوى اه صحي وحقيقي عن السلطة

Real Output     : الاراضي داخل التنظيم اقل بكثير من
Predicted Output: الاراضي داخل التنظيم اقل بكثير من

Real Output     : زي اي شقه لكن موجوده تحت مستوى منسوب
Predicted Output: زي اي شقه لكن موجوده تحت مستوى منسوب الشعب

Real Output     : الدخل فاكيد بتوجهه لحلول التمويل عننا
Predicted Output: الدخل فاكيد بتوجه ل حلول التمويل اا عننا

Real Output     : الخدمات المرتبطه بالعقار قانونيه لي
Predicted Output: خدمات المرتبطه بالعقار قانونيه للي

Real Output     : هيك النسبه يعني هذا الارقام عاليه جدا
Predicted Output: هيك النسب

In [48]:
# import gc
# import numpy as np
# from tqdm import tqdm
# from torch.utils.data import DataLoader
# from transformers.models.whisper.english_normalizer import BasicTextNormalizer
# import torch

# # Define language and task
# language = 'Arabic'
# task = 'transcribe'

# # Assuming `atc_dataset_valid` and `data_collator` are defined somewhere earlier in the code
# eval_dataloader = DataLoader(atc_dataset_valid, batch_size=8, collate_fn=data_collator)

# # Get forced decoder ids for the specific task and language
# forced_decoder_ids = processor.get_decoder_prompt_ids(language=language, task=task)

# # Initialize normalizer
# normalizer = BasicTextNormalizer()

# # Prepare lists to collect predictions and references
# predictions = []
# references = []
# normalized_predictions = []
# normalized_references = []

# # Set model to evaluation mode
# model.eval()

# # Specify the device
# device = torch.device("cuda:0")  # Use GPU 0 (or adjust if needed)
# model.to(device)  # Move model to the chosen device

# # Iterate over the validation dataset
# for step, batch in enumerate(tqdm(eval_dataloader)):
#     with torch.cuda.amp.autocast():  # Automatic mixed precision for inference
#         with torch.no_grad():  # Disable gradient computation for inference
#             # Move input features and labels to the same device as the model (cuda:0)
#             input_features = batch["input_features"].to(device)
#             labels = batch["labels"].to(device)

#             # Generate tokens from the model
#             generated_tokens = (
#                 model.generate(
#                     input_features=input_features,
#                     forced_decoder_ids=forced_decoder_ids,
#                     max_new_tokens=255,
#                 )
#                 .cpu()  # Move the generated tokens to CPU
#                 .numpy()  # Convert to numpy for further processing
#             )
            
#             # Convert labels to numpy and replace -100 with pad_token_id
#             labels = labels.cpu().numpy()
#             labels = np.where(labels != -100, labels, processor.tokenizer.pad_token_id)

#             # Decode the predictions and labels into text
#             decoded_preds = processor.tokenizer.batch_decode(generated_tokens, skip_special_tokens=True)
#             decoded_labels = processor.tokenizer.batch_decode(labels, skip_special_tokens=True)

#             # Extend the lists with the decoded results
#             predictions.extend(decoded_preds)
#             references.extend(decoded_labels)

#             # Normalize the predictions and references
#             normalized_predictions.extend([normalizer(pred).strip() for pred in decoded_preds])
#             normalized_references.extend([normalizer(label).strip() for label in decoded_labels])

#         # Free up memory by deleting unnecessary variables and invoking garbage collection
#         del generated_tokens, labels, batch
#     gc.collect()

# # Compute Word Error Rate (WER) and Normalized WER
# wer = 100 * metric.compute(predictions=predictions, references=references)
# normalized_wer = 100 * metric.compute(predictions=normalized_predictions, references=normalized_references)

# # Store evaluation metrics
# eval_metrics = {"eval/wer": wer, "eval/normalized_wer": normalized_wer}

# # Print the evaluation results
# print(f"{wer=} and {normalized_wer=}")
# print(eval_metrics)
